In [ ]:
#pip3 install pandas primer3-py pyfaidx biopython networkx matplotlib numpy scipy
import pandas as pd
from os import system
import pickle
import random

from time import sleep
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from glob import glob
from os import remove, mkdir

import primer3
from pyfaidx import Fasta

import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

fastq = 'sample.fastq' # Input fastq file name
quality = 0.95 # Define minimal quality Phred score
minSize = 50 # Define the minimum size required for a sequenced read to be utilized in the de novo assembly process, after trimming its ends if necessary
minOverlap = 20 # Define minimal overlap required between two different sequences to be De-Novo assembled together

# Phred quality score
phredict = {'!': 1.0,
 '"': 0.794328,
 '#': 0.630957,
 '$': 0.501187,
 '%': 0.398107,
 '&': 0.316228,
 "'": 0.251189,
 '(': 0.199526,
 ')': 0.158489,
 '*': 0.125893,
 '+': 0.1,
 ',': 0.079433,
 '-': 0.063096,
 '.': 0.050119,
 '/': 0.039811,
 '0': 0.031623,
 '1': 0.025119,
 '2': 0.019953,
 '3': 0.015849,
 '4': 0.012589,
 '5': 0.01,
 '6': 0.007943,
 '7': 0.00631,
 '8': 0.005012,
 '9': 0.003981,
 ':': 0.003162,
 ';': 0.002512,
 '<': 0.001995,
 '=': 0.001585,
 '>': 0.001259,
 '?': 0.001,
 '@': 0.000794,
 'A': 0.000631,
 'B': 0.000501,
 'C': 0.000398,
 'D': 0.000316,
 'E': 0.000251,
 'F': 0.0002,
 'G': 0.000158,
 'H': 0.000126,
 'I': 0.0001,
 'J': 7.9e-05,
 'K': 6.3e-05}

# Extracting high quality sequences from the input fastq file
n = 0
open('sequences.txt', 'w').write('')
with open(fastq, 'r', encoding = 'utf-8-sig') as f:
    current_sequence = ''
    for line in f:
        n += 1
        if n == 2:
            current_sequence = line.strip()
        if n == 4:
            phreds = [phredict[i] for i in line.strip()]
            df = pd.DataFrame()
            df['Sequence'] = [i for i in current_sequence]
            df['Phred'] = phreds
            high_quality_sequence = ''.join(df[df['Phred'] <= quality]['Sequence'].tolist())
            if high_quality_sequence not in current_sequence:
                continue
            elif len(high_quality_sequence) < minSize:
                continue
            else:
                open('sequences.txt', 'a').write(high_quality_sequence + '\n')
            n = 0

sequences = [i.strip() for i in list(set(open('sequences.txt', 'r').read().split('\n'))) if len(i) > 2]
sorted(sequences)
open('sequences.txt', 'w').write('\n'.join(sequences))
total_len = sum([len(i) for i in sequences])
As = round(sum([i.count('A') for i in sequences]) / total_len * 100, 2)
Ts = round(sum([i.count('T') for i in sequences]) / total_len * 100, 2)
Cs = round(sum([i.count('C') for i in sequences]) / total_len * 100, 2)
Gs = round(sum([i.count('G') for i in sequences]) / total_len * 100, 2)

In [ ]:
#Alignment and assembly methods
def right_overlap(genome, sequence):
    overlaps = []
    for n in range(len(sequence) + 1):
        if genome.endswith(sequence[:n]):
            overlap = sequence[:n]
            if len(overlap) != 0:
                overlaps.append(sequence[:n])
    if len(overlaps) == 0:
        return 0
    else:
        return len(overlaps[-1])

def left_overlap(genome, sequence):
    overlaps = []
    for n in range(1, len(sequence)):
        if genome.startswith(sequence[-n:]):
            overlap = sequence[-n:]
            if len(overlap) != 0:
                overlaps.append(sequence[-n:])
    if len(overlaps) == 0:
        return 0
    else:
        return len(overlaps[-1])

def assemble(n):
    sequences = [i for i in open('sequences.txt', 'r').read().split('\n') if len(i) > minOverlap]
    print(n + 1)
    #system('echo ' + str(n))
    genome = sequences[n]
    goback_genome = []
    goback_tracker = 0
    shortening_tracker = 0
    track_len_sequences = [0]
    max_tries_for_sequence = 5
    genome_with_junctions = genome
    assemblies = []
    max_tries = len(sequences)
    tries = 0
    while (len(sequences) != 0) or (tries != max_tries):
        tries += 1
        if goback_tracker > 0:
            goback_tracker += -1
        track_len_sequences.append(len(sequences))
        if (len(track_len_sequences) == max_tries_for_sequence):
            if len(set(track_len_sequences)) == 1:
                try:
                    genome = goback_genome[-1]
                    goback_genome = goback_genome[:-1]
                    goback_tracker += 2
                    if goback_tracker > 10:
                        break
                except:
                    break
            track_len_sequences = [len(sequences)]
        
        maxLeft = 0
        maxLeftSequence = ''
        
        for sequence in sequences:
            if sequence in genome:
                sequences.remove(sequence)
            else:
                lefts = [sequence, sequence[::-1], sequence.replace('A','t').replace('C','g').replace('T','a').replace('G','c').upper(), sequence.replace('A','t').replace('C','g').replace('T','a').replace('G','c').upper()[::-1]]
                for left in lefts:
                    left = left_overlap(genome, sequence)
                    if left > maxLeft:
                        maxLeft = left
                        maxLeftSequence = sequence
        if maxLeft >= minOverlap:
            genome = maxLeftSequence[:-maxLeft] + genome
            genome_with_junctions = maxLeftSequence[:-maxLeft] + '-' + genome_with_junctions
            sequences.remove(maxLeftSequence)
        
        maxRight = 0
        maxRightSequence = ''
    
        for sequence in sequences:
            if sequence in genome:
                sequences.remove(sequence)
            else:
                rights = [sequence, sequence[::-1], sequence.replace('A','t').replace('C','g').replace('T','a').replace('G','c').upper(), sequence.replace('A','t').replace('C','g').replace('T','a').replace('G','c').upper()[::-1]]
                for right in rights:
                    right = right_overlap(genome, sequence)
                    if right > maxRight:
                        maxRight = right
                        maxRightSequence = sequence
        if maxRight >= minOverlap:
            goback_genome.append(genome)
            genome = genome + maxRightSequence[maxRight:]
            genome_with_junctions = genome_with_junctions + '-' + maxRightSequence[maxRight:]
            sequences.remove(maxRightSequence)
        assemblies.append([genome, genome_with_junctions])

        if len(assemblies) > 1:
            if len(assemblies[-1][0]) > len(assemblies[-2][0]):
                shortening_tracker = 0
            else:
                shortening_tracker += 1
            if shortening_tracker == 10:
                break
    
    # remove subsequences
    if len(assemblies) != 0:
        seqs = [i[0] for i in assemblies]
        keep = []
        for n, seq1 in enumerate(seqs):
            count = 0
            for seq2 in seqs:
                if seq1 in seq2 and seq1 != seq2:
                    count += 1
                if count == 2:
                    break
            if count < 2:
                keep.append(n)
        assemblies = [assemblies[i] for i in keep]

    # remove duplicates
    if len(assemblies) != 0:
        df = pd.DataFrame()
        df['genome'] = [i[0] for i in assemblies]
        df['genome_with_junctions'] = [i[1] for i in assemblies]
        df.drop_duplicates(subset=['genome'], keep = 'first', inplace = True)
    return df

#generating results dataframe, removing duplicares and genomes that are sub-strings of other genomes
filtered_sequences = [i for i in open('sequences.txt', 'r').read().split('\n') if len(i) > minOverlap]
print(f'Assembling {len(filtered_sequences)} sequences') 
assemblies = [assemble(n) for n in range(len(filtered_sequences))]
df = pd.concat(assemblies).reset_index(drop = True).drop_duplicates(subset = ['genome'], keep = 'first')
df.sort_values(by = 'genome', key = lambda x: x.str.len(), ascending = False, inplace = True)
df = df[df['genome'].apply(lambda x : len(x) > 3 * len(filtered_sequences[0]))]
gs = df['genome'].tolist()
dups = []
for n,g in enumerate(gs):
    if n == 0 or g not in gs[n-1]:
        dups.append(n)
df = df.iloc[dups].reset_index(drop=True)
df.to_csv('result.csv', index = False)

In [ ]:
# NCBI BLAST search for in-silico validation
df  = pd.read_csv('result.csv').head()
n = 0
def blastn_search(seq):
    global n
    n += 1
    print(n)
    try:
        result_handle = NCBIWWW.qblast('blastn', 'nt', seq)
        sleep(10)
        result = result_handle.read()
        output_file = result.split('<BlastOutput_query-ID>Query_')[1].split('<')[0] + '_blast_output.xml'
        open(output_file, 'w').write(result)
        with open(output_file, 'r') as blast_output:
            blast_records = NCBIXML.parse(blast_output)
            for blast_record in blast_records:
                for alignment in blast_record.alignments:
                    for hsp in alignment.hsps:
                        title = alignment.title
                        identity = round(hsp.match.count('|')/len(hsp.query) * 100, 2)
                        e_value = round(hsp.expect, 5)
                        return (title, identity, e_value)
    except:
        return ('','','')
df['BLASTn'] = df['genome'].apply(blastn_search)
for output_file in glob('*_blast_output.xml'):
    remove(output_file)
df['BLASTn-Result'] = df['BLASTn'].str[0]
df['BLASTn-Identity_(%)'] = df['BLASTn'].str[1]
df['BLASTn-E_value'] = df['BLASTn'].str[2]
df.to_csv('result.csv', index = False)

In [ ]:
# Generating Sanger sequencing assays for molecular validation
def calculatePrimers(sequence):
    primers = primer3.design_primers(
        {
            'SEQUENCE_ID': 'an_id',
            'SEQUENCE_TEMPLATE': sequence,
            'SEQUENCE_TARGET' : [350,100]
        },
        {
            'PRIMER_TASK': 'generic',
            'PRIMER_PICK_LEFT_PRIMER': 1,
            'PRIMER_PICK_INTERNAL_OLIGO': 0,
            'PRIMER_PICK_RIGHT_PRIMER': 1,
            'PRIMER_NUM_RETURN': 10,
            'PRIMER_OPT_SIZE': 20,
            'PRIMER_MIN_SIZE': 18,
            'PRIMER_MAX_SIZE': 27,
            'PRIMER_OPT_TM': 58.0,
            'PRIMER_MIN_TM': 57.0,
            'PRIMER_MAX_TM': 63.0,
            'PRIMER_MIN_GC': 20.0,
            'PRIMER_MAX_GC': 80.0,
            'PRIMER_MAX_POLY_X': 5,
            'PRIMER_SALT_MONOVALENT': 50.0,
            'PRIMER_DNA_CONC': 50.0,
            'PRIMER_MAX_NS_ACCEPTED': 0,
            'PRIMER_MAX_SELF_ANY': 12,
            'PRIMER_MAX_SELF_END': 8,
            'PRIMER_PAIR_MAX_COMPL_ANY': 12,
            'PRIMER_PAIR_MAX_COMPL_END': 8,
            'PRIMER_PRODUCT_SIZE_RANGE': [[100,800]]
        }
    )
    return primers

def generatePrimers(genome):
    header = ['Primer left', 'Primer right', 'Primer left Tm (degrees Celcius)', 'Primer right Tm (degrees Celcius)', 'product_size (bp)', 'Product sequence']
    lines = []
    for i in range(0, len(genome) - 400, 400):
        sequence = genome[i : i + 800]
        try:
            primers = calculatePrimers(sequence) 
            primer_left = primers['PRIMER_LEFT_1_SEQUENCE']
            primer_right = primers['PRIMER_RIGHT_1_SEQUENCE']
            primer_left_Tm = str(int(primers['PRIMER_LEFT_1_TM']))
            primer_right_Tm = str(int(primers['PRIMER_RIGHT_1_TM']))
            product_size = str(int(primers['PRIMER_PAIR_1_PRODUCT_SIZE']))
            primer_right_flipped = primer_right.split(' ')[-1][::-1].replace('A','t').replace('T','a').replace('C','g').replace('G','c').upper()
            product_sequence = primer_left.split(' ')[-1] + sequence.split(primer_left.split(' ')[-1])[1]
            product_sequence = product_sequence.split(primer_right_flipped)[0] + primer_right_flipped
            line = [primer_left, primer_right, primer_left_Tm, primer_right_Tm, product_size, product_sequence]
            lines.append(line)
        except:
            continue
    return lines

df['PCR_validation_settings'] = df['genome'].apply(generatePrimers)
df.to_pickle('results.pkl')

In [ ]:
#Report generation

# Nucleotide type distribution
def nucPercentage(genome, As, Ts, Cs, Gs, output_filename):
    gAs = round(genome.count('A') / len(genome) * 100, 2)
    gTs = round(genome.count('T') / len(genome) * 100, 2)
    gCs = round(genome.count('C') / len(genome) * 100, 2)
    gGs = round(genome.count('G') / len(genome) * 100, 2)
    labels = ['A', 'T', 'C', 'G']
    sequence_1 = [As, Ts, Cs, Gs]
    sequence_2 = [gAs, gTs, gCs, gGs]
    
    barWidth = 0.35
    r1 = np.arange(len(sequence_1))
    r2 = [x + barWidth for x in r1]
    bars1 = plt.bar(r1, sequence_1, width = barWidth, color = 'blue', alpha = 0.7, label = 'Raw data')
    bars2 = plt.bar(r2, sequence_2, width = barWidth, color = 'red', alpha = 0.7, label = 'Assembly')
    
    plt.xlabel('Nucleotides %')
    plt.ylabel('Count')
    plt.title('Comparison of Nucleotide Percentage')
    
    for bar in bars1 + bars2:
        yval = str(bar.get_height())
        if len(yval) == 2:
            yval += '.'
        while len(yval) != 5:
            yval += '0'
        plt.text(bar.get_x() + bar.get_width()/2, float(yval) + 0.5, yval + '%', ha = 'center', va = 'bottom')
    plt.gca().axes.get_yaxis().set_visible(False)
    plt.gca().spines['left'].set_visible(False)
    for spine in plt.gca().spines.values():
        spine.set_visible(False)
    plt.xticks([r + barWidth/2 for r in range(len(sequence_1))], labels)
    plt.legend()
    plt.tight_layout()
    plt.savefig(output_filename, format = 'svg')
    plt.clf()

# De-Bruijn graph generation
def de_bruijn(sequence, k, output_filename):
    edges = []
    for i in range(len(sequence) - k + 1):
        edges.append((sequence[i:i+k-1], sequence[i+1:i+k]))
    G = nx.DiGraph(edges)
    pos = nx.spring_layout(G, k = 0.2)
    nx.draw(G, pos, with_labels = True, node_size = 10, node_color = 'skyblue', font_size = 1, edge_color = "gray", width = 0.1, arrowsize = 0.1)
    plt.title('De Bruijn Graph')
    plt.savefig(output_filename, format = 'svg')
    plt.clf()

# HTML functions
def P(text):
    return '<P>' + text + '</P>'

def H3(text):
    return '<H3>' + text + '</H3>'

# Report output file generation
n = 0
def generateReport(data):
    global n
    n += 1
    genome = data['genome']
    genome_with_junction = data['genome_with_junctions']
    BLASTn_Result = data['BLASTn-Result']
    BLASTn_Identity = data['BLASTn-Identity_(%)']
    BLASTn_E_value = data['BLASTn-E_value']
    PCRs = data['PCR_validation_settings']
    path = 'Results/Prediction_' + str(n)
    fasta = '>Prediction_' + str(n) + '\n' + '\n'.join(genome[i:i+70] for i in range(0, len(genome), 70))
    fasta_with_junctions = '>Prediction_' + str(n) + '_with_alignment_junctions\n' + '\n'.join(genome_with_junction[i:i+70] for i in range(0, len(genome), 70))
    try:
        mkdir(path)
        mkdir(path + '/assets')
    except:
        0
    open(path + '/Prediction_' + str(n) + '.fasta', 'w').write(fasta)
    Fasta(path + '/Prediction_' + str(n) + '.fasta')
    open(path + '/Prediction_' + str(n) + '_with_junctions.fasta', 'w').write(fasta_with_junctions)
    Fasta(path + '/Prediction_' + str(n) + '_with_junctions.fasta')
    k = minSize
    de_bruijn_path = path + '/assets/Prediction_' + str(n) + '_de_bruijn.svg'
    de_bruijn(genome, k, de_bruijn_path)
    nucPrecPath = path + '/assets/Prediction_' + str(n) + '_nucleotide_Percentages.svg'
    nucPercentage(genome, As, Ts, Cs, Gs, nucPrecPath)
    html = '<HTML>\n'
    html += '''<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>tab_title</title>
    <style>
        p.justified {
            text-align: justify;
            max-width: 500px;  /* This is just to make sure the paragraph is not too wide for demonstration purposes. */
        }
        body {
            background-color: white;
        }
    </style>
</head>
<body>
'''.replace('tab_title', 'Prediction_' + str(n))
    html += H3('Sequences') + '\n'
    html += P(fasta.replace('\n', '<br>')) + '\n'
    html += '\n<br>\n'
    html += P(fasta_with_junctions.replace('\n', '<br>')) + '\n'
    html += '\n<br>\n'
    html += H3('BLAST Results') + '\n'
    html += P('Most similar sequence: ' + BLASTn_Result) + '\n'
    html += P('% Identity = ' + str(BLASTn_Identity)) + '\n'
    html += P('E-value = ' + str(BLASTn_E_value)) + '\n'
    html += '\n<br>\n'
    html += H3('Recommended PCRs for valication:') + '\n'
    for PCR in PCRs:
        html += P('Primer forward: ' + PCR[0] + ', Tm = ' + PCR[2] + '&deg;C') + '\n'
        html += P('Primer Reverse: ' + PCR[1] + ', Tm = ' + PCR[3] + '&deg;C') + '\n'
        html += P('Product length = ' + PCR[4] + 'bp') + '\n'
        html += P('Product sequence:') + '\n'
        product = '\n'.join(PCR[5][i:i+70] for i in range(0, len(PCR[5]), 70))
        html += P(product.replace('\n', '<br>')) + '\n'
        html += '\n<br>\n'
    html += '<img src=' + 'Prediction_' + str(n) + '/assets/Prediction_' + str(n) + '_nucleotide_Percentages.svg' + ' alt="Per-nucleotide statistics FASTQ vs. Assembly">' + '\n'
    html += '<img src=' + 'Prediction_' + str(n) + '/assets/Prediction_' + str(n) + '_de_bruijn.svg' + ' alt="De Bruijn Graph">' + '\n'
    html += '\n</body>' + '\n'
    html += '\n</HTML>'
    open(path + '_Summary.html', 'w').write(html)
try:
    remove('Results')
except:
    0
try:
    mkdir('Results')
except:
    0
df.apply(generateReport, axis = 1)